In [ ]:
!pip install mediapipe


In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import time
import json
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, ReLU, Dropout, GRU, ConvLSTM2D, Conv3D, Flatten
from tensorflow.keras.utils import to_categorical
import random 
from sklearn.model_selection import train_test_split
import mediapipe as mp
import pickle as pk

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return  results

In [ ]:
def extract_keypoints(results):

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.25, min_tracking_confidence=0.25) 


In [ ]:
def videoProc(path):
    "###"
    cap = cv2.VideoCapture(path)
    success, frame = cap.read()
    
    fno = 0
    framecount = 20
    fps = round(cap.get(cv2.CAP_PROP_FPS), 0) 
    
    v = []
    


    while True:
        success, frame = cap.read()
        if fno >=framecount:
            break 
        
     

        if success:
            frame = cv2.resize(frame, (720,480), interpolation = cv2.INTER_AREA)
            results = mediapipe_detection(frame, holistic)
            eres = extract_keypoints(results)

            tmp=  np.reshape(eres, (1,258))
            v.append(tmp)
            
            # draw_styled_landmarks(frame, results)
            # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # plt.show()
            fno += 1
        else:
            break

# if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
    cap.release()
    cv2.destroyAllWindows()
    
    f = len(v)
    v = np.array(v) if f >0 else[]
    v =np.reshape(v,(f,225))if f >0 else[]
   


    if f>0:      
        m = v.mean(axis=-1)     
        tmpv = v.T 
        tmpv -=  m
        v = tmpv.T
        m = v.mean(axis=0) 
        v -= m
    if fno<framecount and fno>0:
        tmp = np.zeros((framecount-fno,225))
        v = np.concatenate((v, tmp), axis=0) 

    return v

In [ ]:
v = videoProc("./testdata/z20.mp4")


In [ ]:
v[17].min(),v[17].max()

In [ ]:
reconstructed_model = tf.keras.models.load_model("./modelAsl/v3")

In [ ]:
model.summary()

In [ ]:
p = model.predict_step(np.reshape(v, (1,20,258)))

In [ ]:
np.argmax(p),p